In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

In [2]:
train_data = pd.read_csv('./train.csv')
test_data = test = pd.read_csv('./test.csv')

In [3]:
train_data.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [4]:
test_data.head()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S


In [5]:
train_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB


In [6]:
delet_col = ['Name', 'Cabin', 'Ticket']
train_data_new = train_data.drop(columns = delet_col)
test_data_new = test_data.drop(columns = delet_col)

In [8]:
train_data_new.head()

,PassengerId,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,1,0,3,male,22.0,1,0,7.2500,S
1,2,1,1,female,38.0,1,0,71.2833,C
2,3,1,3,female,26.0,0,0,7.9250,S
3,4,1,1,female,35.0,1,0,53.1000,S
4,5,0,3,male,35.0,0,0,8.0500,S


In [9]:
def zapoln_propuski(x):
    x_copy = x.copy()
    
    # заполнение проусков в категориальных переменных (самым частым значением в колонке)
    cat_col = list(x_copy.select_dtypes(include = ['object']).columns.values)
    for i in cat_col:
        if (x_copy[i].isnull().values.any()):
            x_copy[i].fillna(x_copy[i].mode()[0], inplace = True)
    
    # заполнение пропусков в кол-ых переменных (медианным значением в колонке)
    num_col = list(x_copy.select_dtypes(include = ['int64', 'float64']).columns.values)
    for i in num_col:
        if (x_copy[i].isnull().values.any()):
            x_copy[i].fillna(x_copy[i].median(), inplace = True)
    
    return x_copy

In [10]:
train = zapoln_propuski(train_data_new)
test =  zapoln_propuski(test_data_new)

In [11]:
X_train_gg = train.drop(columns='Survived')
y_train = train['Survived'][:-1]
X_test_gg = test.copy()

In [12]:
X_train_gg

,PassengerId,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,1,3,male,22.0,1,0,7.2500,S
1,2,1,female,38.0,1,0,71.2833,C
2,3,3,female,26.0,0,0,7.9250,S
3,4,1,female,35.0,1,0,53.1000,S
4,5,3,male,35.0,0,0,8.0500,S
...,...,...,...,...,...,...,...,...
886,887,2,male,27.0,0,0,13.0000,S
887,888,1,female,19.0,0,0,30.0000,S
888,889,3,female,28.0,1,2,23.4500,S
889,890,1,male,26.0,0,0,30.0000,C


# One-hot-encoding

In [13]:
def encode(train_data, test_data):
    test_start_ind = train_data.index[-1]
    concated_data = pd.concat([train_data, test_data], sort = False)
    encoded_data = pd.get_dummies(concated_data, drop_first = True)
    data_train_encod, data_test_encod = encoded_data.iloc[:test_start_ind, :], encoded_data.iloc[test_start_ind:, :]
    return data_train_encod, data_test_encod

In [14]:
X_train, X_test = encode(X_train_gg, X_test_gg)

In [15]:
X_train.columns

Index(['PassengerId', 'Pclass', 'Age', 'SibSp', 'Parch', 'Fare', 'Sex_male',
       'Embarked_Q', 'Embarked_S'],
      dtype='object')

In [16]:
from catboost import CatBoostClassifier
from matplotlib import pyplot as plt
from sklearn.compose import ColumnTransformer
from sklearn.decomposition import PCA
from sklearn.ensemble import RandomForestClassifier, StackingClassifier
from sklearn.linear_model import LogisticRegression, LogisticRegressionCV
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.neighbors import KNeighborsClassifier
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.pipeline import make_pipeline
from sklearn.tree import DecisionTreeClassifier, plot_tree

In [17]:
X_train.shape

(890, 9)

In [18]:
y_train.size

890

# RandomForest

In [19]:
PARAMS = {
    "min_samples_split": [9], 
    "min_samples_leaf": [18], 
}
clf = RandomForestClassifier(n_estimators=200, random_state=42)

grid_search = GridSearchCV(clf, PARAMS, scoring="roc_auc", cv=5)
rf = grid_search.fit(X_train, y_train);

In [20]:
print(f"Best score: {rf.best_score_}")
print(f"Best params: {rf.best_params_}")

Best score: 0.8592011565477625
Best params: {'min_samples_leaf': 18, 'min_samples_split': 9}


# Попытка залить 1

In [36]:
predictions = rf.predict(X_test)[1:]

output = pd.DataFrame({'PassengerId':test_data.PassengerId, 'Survived':predictions})
output.to_csv('submission.csv',index=False)

In [21]:
def make_prediction_file(predictions, test_data, name):
    result = pd.DataFrame({'PassengerId': test_data.reset_index()['PassengerId'], 'Survived': predictions})
    result.to_csv(name, index = False)

In [ ]:
make_prediction_file(y_pred, test, 'titanic_first_proba.csv')

# LogisticRegressionCV

In [28]:
std_scaler = StandardScaler()
log_reg_cv = LogisticRegressionCV(scoring='roc_auc')

cv = GridSearchCV(log_reg_cv, 
                  param_grid={
                      'penalty': ['l1', 'l2'],
                      'solver': ['lbfgs', 'newton-cg', 'liblinear', 'sag', 'saga'],
                  },
                  scoring='roc_auc',
                  refit=True,
                  n_jobs=-1,
                  verbose=0
)

lrcv = cv.fit(X_train, y_train)

C:\Users\Bulat\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:372: FitFailedWarning: 
15 fits failed out of a total of 50.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
5 fits failed with the following error:
Traceback (most recent call last):
  File "C:\Users\Bulat\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py", line 680, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "C:\Users\Bulat\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py", line 2031, in fit
    solver = _check_solver(self.solver, self.penalty, self.dual)
  File "C:\Users\Bulat\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py", line 447, in _check_solver
    raise ValueError(
Value

In [29]:
print(f"best score: {lrcv.best_score_}")
print(f"best params: {lrcv.best_params_}")

best score: 0.8493519820083091
best params: {'penalty': 'l1', 'solver': 'liblinear'}


# Knn

In [30]:
# numeric_transformer = Pipeline(steps=[
#     ("scaler", StandardScaler())
# ])
# preprocessor = ColumnTransformer(
#     transformers=[
#         ("num", numeric_transformer, num_cols),
#         ("cat", OneHotEncoder(handle_unknown='ignore'), cat_cols)
#     ]
# )
PARAMS = {
    "model__n_neighbors": [44],  
    "model__metric": ["manhattan"],  
}

clf = Pipeline(steps=[
    ("model", KNeighborsClassifier()),
])
    
grid_search = GridSearchCV(
    estimator=clf,
    param_grid=PARAMS,
    scoring="roc_auc",
    n_jobs=-1,
    cv=10,
    refit=True,
)
knn = grid_search.fit(X_train, y_train)

In [31]:
print(f"best score: {knn.best_score_}")
print(f"best params: {knn.best_params_}")

best score: 0.5991535806241688
best params: {'model__metric': 'manhattan', 'model__n_neighbors': 44}


In [32]:
meta = CatBoostClassifier(
    logging_level='Silent',
    eval_metric="AUC:hints=skip_train~false",
    metric_period=1000,
    random_seed=0,
    grow_policy="Depthwise",
    l2_leaf_reg=1,
    learning_rate=0.08,
    max_depth=10,
    min_data_in_leaf=10,
    n_estimators=10,
    random_strength=11,
    subsample=0.1,
)
stacking = StackingClassifier(
    estimators=[
        ("knn", knn),
        ("rf", rf),
        ("lrcv", lrcv),
    ],
    final_estimator=meta,
    n_jobs=-1,
)

stacking.fit(X_train, y_train)

StackingClassifier(estimators=[('knn',
                                GridSearchCV(cv=10,
                                             estimator=Pipeline(steps=[('model',
                                                                        KNeighborsClassifier())]),
                                             n_jobs=-1,
                                             param_grid={'model__metric': ['manhattan'],
                                                         'model__n_neighbors': [44]},
                                             scoring='roc_auc')),
                               ('rf',
                                GridSearchCV(cv=5,
                                             estimator=RandomForestClassifier(n_estimators=200,
                                                                              random_state=42),
                                             param_grid={'min_samples_leaf': [18],
                                                         'min_sample

In [33]:
best_model = stacking

In [41]:
predictions = best_model.predict(X_test)[1:]

output = pd.DataFrame({'PassengerId':test_data.PassengerId, 'Survived':predictions})
output.to_csv('submission.csv',index=False)

In [40]:
y_pred = best_model.predict_proba(X_test)[1:, 1]
submission = pd.read_csv('./submission.csv')
submission['Survived'] = y_pred
submission.to_csv("submission.csv", index=False)

In [81]:
def make_prediction_file(predictions, test_data, name):
    result = pd.DataFrame({'PassengerId': test_data.reset_index()['PassengerId'], 'Survived': predictions})
    result.to_csv(name, index = False)

In [84]:
make_prediction_file(y_pred, test, 'titanic_first_proba.csv')